# Procesamiento de imágenes - Fase experimental

## Etapa de procesamiento de imágenes

Durante esta etapa se procesan las imágenes, básicamente se busca hacer la transición de RGB a HSV, hacer el blurring de la imagen, las equalizaciones de histograma y el cambio del modo de color.

In [19]:
import numpy as np
import cv2
# Se carga la imagen
color = cv2.imread("truchas/salmonid.000006.png", 1)
# Se transforma la imagen
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
cv2.imshow('en hsv', hsv)
# Se extraen h s y v de la imagen
h,s,v = cv2.split(hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
#Bluring
# ksize
ksize = (15, 15)
blur_h = cv2.blur(h, ksize)
blur_s = cv2.blur(s, ksize)
blur_v = cv2.blur(v, ksize)

#merge
merge = cv2.merge((blur_h,blur_s,blur_v))
cv2.imshow('en rgb', merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
#Global histogram equalization

equ_v = cv2.equalizeHist(blur_v)

#merge
merge = cv2.merge((blur_h,blur_s,equ_v))
cv2.imshow('en rgb', merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
#CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(9,9))
cl_h = clahe.apply(blur_h)
cl_s = clahe.apply(blur_s)
cl_v = clahe.apply(equ_v)

#merge
merge = cv2.merge((cl_h,cl_s,cl_v))
cv2.imshow('en rgb', merge)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
#Morphological transformation
kernel = np.ones((5,5),np.uint8)

#Opening
op_h = cv2.morphologyEx(cl_h, cv2.MORPH_OPEN, kernel)
op_s = cv2.morphologyEx(cl_s, cv2.MORPH_OPEN, kernel)
op_v = cv2.morphologyEx(cl_v, cv2.MORPH_OPEN, kernel)

#merge
merge = cv2.merge((op_h, op_s, op_v))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# Binary threshold
img = cv2.cvtColor(merge, cv2.COLOR_BGR2GRAY)
cv2.imshow('binario', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('thresh1', thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
# Find contours, obtain bounding box, extract and save ROI
ROI_number = 0
cnts = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(color, (x, y), (x + w, y + h), (36,255,12), 2)
    ROI = color[y:y+h, x:x+w]
    #cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
    ROI_number += 1

cv2.imshow('image', color)
cv2.waitKey()

-1

## Etapa de definición de boxes

In [26]:
# Find contours, obtain bounding box, extract and save ROI
ROI_number = 0
cnts = cv2.findContours(thresh1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(color, (x, y), (x + w, y + h), (36,255,12), 2)
    ROI = color[y:y+h, x:x+w]
    #cv2.imwrite('ROI_{}.png'.format(ROI_number), ROI)
    ROI_number += 1

cv2.imshow('image', color)
cv2.waitKey()

-1

## Para escribir las imágenes

In [27]:
##cv2.imwrite(filename, image)